In [1]:
import os
import sys
import json
from time import sleep
import datetime
import numpy as np
import nibabel as nib
import scipy
from skimage.filters import threshold_triangle as triangle
from sklearn.preprocessing import quantile_transform
from skimage.filters import unsharp_mask

In [2]:
def clean_anat(in_file, save_dir):
	### Load brain ###
	#file = os.path.join(directory, 'stitched_brain_red_mean.nii') 
	brain = np.asarray(nib.load(in_file).get_data(), dtype='float32')

	### Blur brain and mask small values ###
	print('masking')
	brain_copy = brain.copy().astype('float32')
	brain_copy = scipy.ndimage.filters.gaussian_filter(brain_copy, sigma=10)
	threshold = triangle(brain_copy)
	brain_copy[np.where(brain_copy < threshold/2)] = 0

	### Remove blobs outside contiguous brain ###
	print('removing blobs')
	labels, label_nb = scipy.ndimage.label(brain_copy)
	brain_label = np.bincount(labels.flatten())[1:].argmax()+1
	brain_copy = brain.copy().astype('float32')
	brain_copy[np.where(labels != brain_label)] = np.nan

	### Perform quantile normalization ###
	print('quantile_norm')
	brain_out = quantile_transform(brain_copy.flatten().reshape(-1, 1), n_quantiles=500, random_state=0, copy=True)
	print('1')
	brain_out = brain_out.reshape(brain.shape)
	print('2')
	np.nan_to_num(brain_out, copy=False)

	### Save brain ###
	print('saving')
	fname = in_file.split('/')[-1].split('.')[0]
	save_file = os.path.join(save_dir, f'{fname}_clean.nii')
	aff = np.eye(4)
	img = nib.Nifti1Image(brain_out, aff)
	img.to_filename(save_file)

In [3]:
main_dir = "/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20210126_alignment_package"
seed_brain = ""

##########################
### 1) Clean Anatomies ###
##########################
# Loop over each anatomy in "raw_anats" directory, and saved a cleaned version to "clean_anats" directory
raw_dir = os.path.join(main_dir, 'raw_anats')
anats = os.listdir(raw_dir)
print('found raw anats: {}'.format(anats))

save_dir = os.path.join(main_dir, 'clean_anats')
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

print('start cleaning')
clean_anat(os.path.join(raw_dir, anats[0]), save_dir)
print('finished cleaning')

found raw anats: ['fly_089_anat_raw.nii', 'fly_106_anat_raw.nii', 'fly_105_anat_raw.nii', 'fly_092_anat_raw.nii', 'fly_110_anat_raw.nii', 'fly_102_anat_raw.nii', 'fly_087_anat_raw.nii', 'fly_096_anat_raw.nii', 'fly_098_anat_raw.nii', 'fly_094_anat_raw.nii', 'fly_099_anat_raw.nii', 'fly_093_anat_raw.nii', 'fly_111_anat_raw.nii', 'fly_101_anat_raw.nii', 'fly_097_anat_raw.nii', 'fly_100_anat_raw.nii']
start cleaning
masking
removing blobs
quantile_norm
1
2
saving
finished cleaning
